# Exercice d'introduction à PostGIS

PostGIS est une extension SIG pour PostgreSQL, une puissante base de données relationnelle. Dans cet exercice, nous allons apprendre les choses suivantes:

1. Installation de PostgreSQL et PostGIS
- Interaction avec PostgreSQL en mode ligne de commande
- Installation d'une interface graphique pour PostgreSQL (pgAdmin III, PSequel)
- Création d'une base de données PostGIS
- Chargement d'un fichier Shape dans PostGIS. Nous allons charger les communes suisses ainsi que les toponymes de la carte nationale au 1:200'000
- Utilisation de PostGIS avec QGIS
- Interrogation de la base de données en SQL
- Création d'une nouvelle couche des cantons suisses, à partir de la couche des communes
- Chargement d'un fichier CSV (avec des données statistiques sur les cantons suisses)
- Jointure des données du fichier CSV avec la couche des cantons
- Calcul du nombre de montagnes par canton, et ajout à la table des données des cantons

Cet exercice est optimisé pour un environnement Linux, mais il peut être utilisé de manière presque inchangé pour Mac OS X et Windows.

Les données utilisées pour l'exercice sont disponibles sur [github.com/interactivethings/swiss-maps](https://github.com/interactivethings/swiss-maps).

**Note importante: Il est important de prendre des notes!** Cet exercice contient une série de commandes passées en Terminal. En même temps que d'exécuter ces commandes, **copiez-les dans un fichier texte, dans Evernote,** ou autre outil du genre. Une bonne organisation va vous faciliter la vie une fois que vous devez refaire les choses (et ça vous fera gagner beaucoup de temps!).


## 1. Installation de PostgreSQL et PostGIS

Sur les ordinateurs de Géopolis, il y a une machine virtuelle Ubuntu 14.04LTS qui donne un accès administrateur à l'ensemble du système. PostGIS y est déjà installé. Pour démarrer la machine virtuelle, il faut lancer d'abord VirtualBox, et puis démarrer la machine virtuelle Ubuntu.

Le [site Web de PostGIS](http://postgis.net/install/) donne des instructions détaillées pour l'ensemble des plateformes. Nous conseillons ici une piste possible parmi d'autres pour l'installation.

### 1.1 Linux (Ubuntu, Debian)

Linux possède un installateur de logiciels déjà intégré. L'installation peut être faite avec les commandes suivantes (`sudo` permet d'exécuter la commande suivante en mode admin aussi appelé `root`):

    sudo apt-get update
    sudo apt-get install postgresql-9.3 postgis postgresql-9.3.-postgis-2.1 postgresql-9.3-postgis-scripts

Attention: la version de PostgreSQL (9.3 ici) ou de PostGIS (2.1 ici) peut évidemment évoluer. Avec la commande `apt-cache search postgresql | grep postgis` on peut lister quelle est la version actuelle des paquets.


### 1.2 Mac OS X

Sur OS X, il y a la possibilité d'installer [Postgres.app](http://postgresapp.com/).

Une autre solution, un peu semblable à la version Linux, est d'utiliser [Homebrew](http://brew.sh/). Homebrew est un installateur de logiciels Unix pour OSX. Il faut donc d'abord installer Homebrew (suivre les instructions sur le site).

Il faudra également installer XCode (disponible dans l'AppStore).

Une fois Homebrew installé, il suffit de passer les commandes suivantes:

    brew update
    brew install postgresql postgis


### 1.3 Windows

Voir les instructions [ici](http://www.bostongis.com/?content_name=postgis_tut01).
    

## 2. Interaction avec PostgreSQL en mode ligne de commande

*Les instructions qui suivent peuvent varier d'une installation à l'autre, en fonction du mode d'installation choisi et des paramètres d'installation. Nous allons donner ici les instructions pour une installation typique sous Ubuntu, tout en essayant d'être aussi général que possible.*

PostgreSQL vient avec plusieurs logiciels de ligne de commande, dont le logiciel principal est **`psql`**. Il s'agit d'une console interactive pour exécuter des requêtes SQL et lancer quelques commandes spécifiques à Postgres.

Il y a également des logiciels pour créer des bases de données (`createdb`) ou utilisateurs (`createuser`), ou pour en supprimer (`dropdb`, `dropuser`). Il y a également des logiciels d'export de bases de données (`pg_dump`, `pg_dumpall`).

Pour commencer, nous allons créer une nouvelle base de données **`intro_postgis`** qui nous appartient (c'est-à-dire à l'utilisateur avec lequel nous nous sommes authenifié au démarrage de l'ordinateur; par la suite nous allons simplement prendre le nom **`user`**, à modifier selon votre propre installation).

*L'étape suivante n'est pas nécessaire sur Windows ou avec l'application Postgres.app.*
Dans une première étape, nous devons créer l'utilisateur `user` dans PostgreSQL. Pour cela, nous devons d'abord obtenir les droits d'admin pour la base de données. Dans la plupart des installations Postgres, il y a un utilisateur système **`postgres`** qui a les droits d'admin par défaut. Ce que nous allons donc faire est de (1) nous authentifier comme utilisateur `postgres`, (2) créer un utilisateur `user` qui est admin (`superuser`) et (3) fermer la connexion en tant qu'utilisateur `postgres`:

    sudo su - postgres
    createuser --superuser user
    exit

Maintenant, nous pouvons simplement créer la base de données:

    createdb intro_postgis

ou si vous êtes déjà dans `psql` (p.ex. sur Windows):

    CREATE DATABASE intro_postgis;

Pour entrer dans la console SQL interactive, il suffit de lancer `psql` sur la nouvelle base de données:

    psql intro_postgis
    
ou si vous êtes déjà dans `psql`:

    \c intro_postgis

Nous pouvons désormais exécuter toute requête SQL, comme par exemple:

    CREATE TABLE test ( id serial primary key, nom varchar(50) );
    INSERT INTO test (nom) VALUES ('hello world');
    SELECT * FROM test;
    
PostgreSQL utilise quelques particularités dans le SQL utilisé (comme par ailleurs toutes les bases de données), dont voici les plus importantes:

- Pour créer une clé primaire sous forme de numéros incrémentiels automatiques, nous devons choisir le type `serial`. Postgres ne connaît pas l'option `auto_increment` disponible dans d'autres bases de données.

- Les chaînes de caractères doivent être incluses entre des apostrophes (`'...'`). Les guillemets ne fonctionnent pas pour cela dans Postgres. Pour écrire un apostrophe dans une chaîne de caractère, il faut écrire deux apostrophes, p.ex. `'L''Orient'`.

- Les guillemets sont réserver pour des noms de tables, champs, bases de données etc. Généralement, nous n'allons pas en avoir besoin, mais dans certains cas où les noms contiennent p.ex. un tiret, oui. Nous pourrions donc écrire la requête de création de table ci-dessous de manière strictement équivalente comme:

        CREATE TABLE "test" ("id" serial primary key, "nom" varchar(50) );

- Les noms des tables, bases de données etc. doivent être considérés comme étant sensibles à la casse dans Postgres. Sur certains OS, ce n'est pas strictement le cas, mais respecter cette limite nous évite des mauvaises surprises. Il est un bonne pratique de n'utiliser que des noms en minuscules et de séparer éventuellement des parties avec un *underscore*, p.ex. écrire plutôt `utilisation_du_sol` et non `UtilisationDuSol`, `Utilisation-Sol` ou autre variante plus exotique.



Le logiciel `psql` propose une aide intégrée très simple. En tapant `help` dans la console, nous avons les instructions nécessaire pour poursuivre notre quête aux informations. Parmi les instructions les plus utiles, on trouve dans l'aide sur les commandes `psql` (taper `\?`) la liste des bases de données (`\l`) ou des tables (`\dt`).


Finalement, pour **sortir de `psql`**, il suffit de taper **`ctrl-D`**.

## 3. Installation d'une interface graphique pour PostgreSQL

### 3.1 pgAdmin III

[**pgAdmin**](http://www.pgadmin.org) est un **client pour Postgres** avec GUI. Il s'agit probablement du client le plus répandu, malgré son interface parfois assez rudimentaire. Sous Ubuntu/Debian, il peut être installé simplement avec `sudo apt-get install pgadmin3`.


### 3.2 PSequel (OS X)

[**PSequel**](http://www.psequel.com/) est une alternative plus conviviale à pgAdmin, pour Mac OS X, mais qui est encore dans les débuts du développement. Le but est de développer un logiciel similaire à [`Sequel Pro`](http://www.sequelpro.com/) qui est uniquement disponible pour MySQL.


### 3.3 phpPgAdmin

[**phpPgAdmin**](http://phppgadmin.sourceforge.net) est un client Web pour PostgreSQL. Il doit tourner sur un serveur Web avec PHP.

Nous n'allons pas aborder cette dernière méthode, car sur notre propre ordinateur, il est plus convival de travailler avec un client Desktop, qui en plus est capable de se connecter à n'importe quelle base de données, même sur un autre ordinateur.


## 4. Création d'une base de données PostGIS.

Une base données PostGIS est tout d'abord une simple base de données PostgreSQL. Mais nous allons activer à l'intérieur de notre base de données PostgreSQL l'extension PostGIS:

    CREATE EXTENSION postgis;

Cette instruction crée une nouvelle table `spatial_ref_sys` utilisée par PostGIS, ainsi que quelques fonctions pour effectuer des opérations de type SIG (en fait, plus de 1000 fonctions...).

Désormais, il nous est possible d'utiliser des colonnes de types `geometry`.



## 5. Chargement d'un fichier Shape dans PostGIS. Nous allons charger les communes suisses ainsi que les toponymes de la carte nationale au 1:200'000.

Avant de charger des données, nous devons copier les données de l'exercice sur notre disque local. Les données de l'exercice sont disponibles sur GitHub et peuvent facilement être téléchargées à l'aide du logiciel `git` dans le Terminal:

    git clone https://github.com/interactivethings/swiss-maps.git

Les données se trouvent ensuite dans un dossier `swiss-maps`.

---

Il y a plusieurs façons de charger des fichiers Shape dans une table PostGIS. Une façon relativement simple est l'outil `shp2pgsql` disponible dans le Terminal. Pour importer notre couche des communes, nous pouvons écrire:

    shp2pgsql -s 21781 -I -W "ISO-8859-1" swiss-maps/src/V200/2014/VEC200_Commune.shp vec200_communes | psql intro_postgis

L'instruction `-s 21781` définit le système de référence spatial (CH1903, code EPSG 21781) utilisé dans le fichier Shape. `-I` crée un *index* sur la colonne des géométries, ce qui permet d'accéler les requêtes spatiales. `-W "ISO-8859-1"` définit l'encodage de la table d'attributs du fichier Shape, pour que les caractères spéciaux (accents) soient correctement représentés dans la base de données. `vec200_communes` est le nom de la table à créer dans PostGIS.

L'instruction un peu bizarre `| psql intro_postgis` redirige la sortie du logiciel `shp2pgsql` vers la base de données `intro_postgis`. L'opérateur shell `|` est appelé *pipe* (en anglais!) et permet d'enchainer des logiciels de lignes de commande. On aurait aussi pu rediriger la sortie de `shp2pgsql` dans un fichier en écrivant `> nom_du_fichier.sql` à la place (attention, cette commande écrase un fichier existant!).

Nous pouvons par la suite procéder de la même manière pour le fichier Shape des toponymes:

    shp2pgsql -s 21781 -I -W "ISO-8859-1" swiss-maps/src/sn200/sn200.shp sn200_toponymes | psql intro_postgis


Pour vérifier si l'import s'est fait correctement, nous pouvons aller dans `psql` (avec `psql intro_postgis` et faire une requête de sélection simple (n'oubliez pas d'insérer `LIMIT 10` à la fin pour limiter le nombre de lignes renvoyées):

    SELECT * FROM sn200_toponymes LIMIT 10;

On peut constater que l'ensemble des attributs a été importé, ainsi qu'une colonne `geom` qui contient un code un peu bizarre. Il s'agit de la représentation interne à PostGIS d'une géométrie. Pour avoir une représentation un peu plus lisible, on peut demander p.ex. d'avoir la géométrie en format WKT (Well-Known Text):

    SELECT ST_AsText(geom) FROM sn200_toponymes LIMIT 10;

Ou encore pour extraire les coordonnées X/Y séparément:

    SELECT ST_X(geom), ST_Y(geom) FROM sn200_toponymes LIMIT 10;

Nous allons discuter plus de fonctions spatiales PostGIS dans la suite de l'exercice.

---


Des **alternatives à `shp2pgsql`** sont le logiciel de ligne de commande `ogr2ogr` (mais la syntaxe d'importation est un peu plus compliquée) ou l'extension QGIS `SPIT` (avec interface graphique, ce qui rend impossible de noter nos commandes dans un fichier texte).

Le logiciel **`ogr2ogr`** est par ailleurs utile dans d'autres contextes, par exemple la conversion simple de fichiers Shape dans un autre format (Mapinfo, GeoJson, ...), ou vice versa.


## 6. Utilisation de PostGIS avec QGIS

QGIS peut lire et écrire des couches PostGIS sans problème. Il suffit d'ouvrir QGIS, et choisir *Layer > Add Layer... > Add PostGIS Layer*. Par la suite il faut se connecter à la base de données (p.ex. `intro_postgis`).

![Paramètres de connexion à PostGIS (gauche) et sélection des couches à charger (droite)](exercice-postgis-figures/qgis-load-postgis-layer.png)

Ensuite, il suffit de choisir les tables à ajouter à QGIS. Une fois que la table PostGIS a été ajoutée au projet QGIS, on peut l'utiliser normalement comme toute autre couche, y compris pour l'édition du contenu.

---

Il se peut que vous receviez **une erreur lors de la connexion à la base de données**, en raison d'un problème d'authentification. En effet, QGIS gère l'accès à PostgreSQL un peu différemment, et nous devons adapter les droits d'accès à la base de données. Le plus simple est de simplement autoriser l'accès à toutes les bases de données PostgreSQL depuis l'ordinateur local. Pour ce faire, il faut modifier le fichier de configuration `pg_hba.conf` de PostgreSQL et ainsi redémarrer le serveur de base de données.

Sur Ubuntu, dans une installation standard avec Postgres 9.3, ce fichier se trouve au chemin d'accès `/etc/postgresql/9.3/main/pg_hba.conf`. Pour l'éditer, nous devons avoir les droits d'admin ou être l'utilisateur `postgres`. Nous pouvons le faire ainsi:

    sudo pico /etc/postgresql/9.3./main/pg_hba.conf

Ceci lance l'éditeur de fichiers `pico`. Il faut maintenant descendre dans le fichier et changer sur les deux lignes ci-dessous la partie `md5` en `trust`:

    # IPv4 local connections:
    host    all             all             127.0.0.1/32            trust
    # IPv6 local connections:
    host    all             all             ::1/128                 trust

Puis il faut sauvegarder le fichier (dans Pico, c'est avec `ctrl-O`) et fermer le fichier (`ctrl-X`).

Pour redémarrer Postgres, il suffit de taper `sudo service postgresql restart`.

---

**Et PostGIS avec ArcGIS?**

ArcGIS peut utiliser les données de PostGIS de deux manière possibles:

1. PostGIS peut être utilisé pour stocker une Geodatabase Enterprise ArcSDE (et nécessite donc ArcGIS Server Enterprise). Toutes les fonctionnalités d'une Geodatabase sont alors supportées. Cette possibilité existe depuis la version 9.3 d'ArcGIS.

2. Avec la version 10 d'ArcGIS, il est possible d'ajouter des *query layers* qui permettent à ArcGIS de faire des connexions SQL à des bases de données spatiales, dont PostGIS. Les données accédées de cette manière peuvent être affichées et analysées dans ArcGIS, par contre elles ne sont pas éditables (accès lecture seule).

## 7. Interrogation de la base de données en SQL

La base de données PostGIS peut être interrogée en utilisant le langage SQL, à l'aide de l'interface ligne de commande, interface graphique ou même par de nombreux langages de programmation. En plus de la simple interrogation, il est possible de créer des tables, les modifier, insérer, mettre à jour ou supprimer des données dans les tables. Bref, le SQL est le langage universel de manipulation de base de données relationnelle.

Le but de cet exercice n'est évidemment pas de faire une introduction complète au SQL, mais uniquement de donner quelques instructions de bases qui peuvent permettre d'exécuter quelques opérations de base.

### 7.1 Sélection de données

Nous pouvons sélectionner l'ensemble des colonnes et lignes dans la table `sn200_toponymes` simplement avec la commande `SELECT`. L'étoile `*` signifie que nous voulons avoir l'ensemble des colonnes:

    SELECT * FROM sn200_toponymes;

Nous pouvons également préciser les colonnes que nous désirons avoir. En plus, il est possible de faire des opérations de tri avec `ORDER BY` suivi du nom du champ:

    SELECT objectval, name FROM sn200_toponymes
    ORDER BY name;
    
Il est également possible de trier en sens inverse avec `ORDER BY name DESC`.

L'instruction `GROUP BY` est intéressante car elle cherche les valeurs uniques dans la colonne spécifiée, et regroupe toutes les lignes ayant la même valeur pour cette colonne. Ainsi, il est possible de rapidement connaître l'ensemble des valeurs possibles pour un champ, comme ici pour le type d'objet auquel le toponyme se réfère:
    
    SELECT objectval FROM sn200_toponymes GROUP BY objectval;

et même de compter le nombre de toponymes pour chaque type d'objet:

    SELECT objectval, COUNT(*) FROM sn200_toponymes GROUP BY objectval;


### 7.2 Insertion de données

    INSERT INTO sn200_toponymes (gid, objectval, name) VALUES (99999, 'Dorigny', 'Université');

### 7.3 Modification de données

    UPDATE sn200_toponymes SET name = 'Mouline` WHERE gid = 99999;

### 7.4 Suppression de donneés

    DELETE FROM sn200_toponymes WHERE name = 'Mouline';
    
Supprimer l'ensemble du contenu d'une table est extrêment simple. En plus d'être simple, il n'y a ni avertissement ni possibilité de revenir en arrière. Et c'est extrêmement efficace et rapide! Voici comment le faire:

    DELETE FROM sn200_toponymes;

### 7.5 Suppression d'une table

Pas compliqué non plus:

    DROP TABLE sn200_toponymes;

Et ça aussi sans avertissement ou possibilité de revenir en arrière. C'est définitif!




## 8. Création d'une nouvelle couche des cantons suisses, à partir de la couche des communes

La couche des cantons peut être calculée à partir des communes grâce à l'attribut qui contient le code du canton pour chaque commune. Dans le langage des SIG, il s'agit d'effectuer un *dissolve* sur la couche des communes, par l'attribut du canton. Dans le langage des bases de données, il s'agit d'une agrégation exprimée par l'instruction SQL `GROUP BY`. Mais avant de passer à cette étape d'agrégation, nous allons supprimer les entités qui ne sont pas strictement nécessaires (notamment les régions à l'étranger). Pour cela, nous pouvons faire une sélection très simple en SQL:

    SELECT * FROM vec200_communes WHERE country = 'CH';

L'instruction `GROUP BY` regroupe toutes les lignes qui ont la même valeur pour le champ fourni. Nous pouvons donc utiliser l'instruction `GROUP BY kantonsnr` (`kantonsnr` contenant le code numérique du canton) pour un regroupement par canton. Il faut encore donner une *fonction d'agrégation* qui permet de calculer les différents champs à partir des communes pour chaque canton. Par exemple, pour connaître le nombre de communes par canton, il suffit d'exécuter la requête suivante:

    SELECT kantonsnr, COUNT(*) 
    FROM vec200_communes 
    WHERE country = 'CH'
    GROUP BY kantonsnr;

La fonction `COUNT` renvoie simplement le nombre de lignes ayant la même valeur pour `kantonsnr`. D'autres fonctions comme `SUM`, `MEAN` etc. existent. Dans PostGIS, il y a une fonction qui s'appelle `ST_UNION` qui permet d'effectuer une fusion de toutes les géométries ayant la même valeur pour le champ donné dans le `GROUP BY`. Du coup, la couche des cantons peut se calculer comme suit:

    SELECT kantonsnr, COUNT(*) AS ncmnes, ST_UNION(geom) AS geom
    FROM vec200_communes
    WHERE country = 'CH'
    GROUP BY kantonsnr;

Notez que nous avons cette fois donné un nouveau nom aux champs calculés, à l'aide de l'instruction `AS alias` où `alias` est le nouveau nom donné.

Pour pérenniser notre nouvelle couche des cantons, on peut créer une table à partir de la requête ci-dessus:

    CREATE TABLE vec200_cantons AS
    SELECT kantonsnr, COUNT(*) AS ncmnes, ST_UNION(geom) AS geom
    FROM vec200_communes
    WHERE country = 'CH'
    GROUP BY kantonsnr;

### Exercice

Cette nouvelle couche des canton comprend les lacs également (p.ex. le canton de Vaud est partiellement localisé dans le Léman). Calculez une nouvelle couche des cantons qui ne comprend pas les lacs.


## 9. Chargement d'un fichier CSV (avec des données statistiques sur les cantons suisses)

Nous devons d'abord créer une nouvelle table qui peut accueillir notre fichier CSV:

    CREATE TABLE cantons_stats (
        geocode INTEGER PRIMARY KEY,
        abbr VARCHAR(2),
        nom VARCHAR(40),
        pop2010 INTEGER,
        pop2014 INTEGER
    );
    
Et maintenant nous pouvons simplement copier notre fichier dans la base de données, depuis `psql`:

    COPY cantons_stats (geocode, abbr, nom, pop2010, pop2014) FROM 'donnees_cantons.txt' DELIMITER ',' CSV;

Il peut être nécessaire de donner l'ensemble du chemin d'accès du fichier, et non seulement le nom comme dans l'exemple ci-dessus.

[Le fichier donnees_cantons.txt](https://raw.githubusercontent.com/christiankaiser/geovis2/master/notebooks/exercice-postgis-data/donnees_cantons.txt)


## 10. Jointure des données du fichier CSV avec la couche des cantons

    SELECT * FROM vec200_cantons C
    JOIN cantons_stats S ON C.kantonsnr = S.geocode;

ou de manière équivalente

    SELECT * FROM vec200_cantons C, cantons_stats S
    WHERE C.kantonsnr = S.geocode;



## 11. Calcul du nombre de montagnes par canton, et ajout à la table des données des cantons


Le problème de compter les sommets par canton est intéressant dans le sens que bien souvent, les sommets se trouvent sur une limite cantonale. Cependant, on regardant le jeu de données de près, on se rend compte que le sommet n'est pas exactement sur la frontière, mais quelques centimètres voir mètres à côté. Voici l'exemple du Grand Muveran qui est localisé entre Vaud et Valais:

![Le Grand Muveran à cheval entre Vaud et Valais](exercice-postgis-figures/qgis-sommet-frontiere.png)

Pour résoudre ce problème, nous allons dans un premier temps sélectionner tous les sommets (où le champ `objectval` se termine sur *Gipfel*) et en même temps nous calculons une zone tampon de 50 mètres autour de chaque sommet:

    SELECT gid, ST_Buffer(geom, 50) AS sommet
    FROM sn200_toponymes
    WHERE objectval LIKE ('%Gipfel');

Sur cette base, nous pouvons nous interroger dans quel canton se localise le sommet, en faisant une jointure entre cette requête et la table des cantons:

    SELECT gid, kantonsnr FROM
    (  SELECT gid, ST_Buffer(geom, 50) AS sommet
       FROM sn200_toponymes
       WHERE objectval LIKE ('%Gipfel')
    ) A
    JOIN vec200_cantons C ON ST_Intersects(A.sommet, C.geom);

Ça nous fait la liste des sommets avec le numéro de canton associé. Un sommet peut s'y trouver deux fois, s'il est localisé entre deux cantons. La fonction `ST_Intersects(geom1, geom2)` renvoie `TRUE` si les 2 géométries s'intersectent (ont une surface non nulle en commun).

Dans cette requête, nous avons utilisé le concept de sous-requête entre parenthèses. Nous devons toujours donner un *alias* (nom) à une sous-requête, et nous avons simplement choisi `A`. Pour la table `vec200_cantons`, nous avons également donné un alias (`C`), chose qui est optionnel cette fois.

Au final, sur la base de cette requête, il suffit désormais de faire une agrégation par canton, sauf que nous pouvons simplement faire la somme pour avoir le nombre de sommets:

    SELECT kantonsnr, COUNT(*) AS n FROM
    (  SELECT gid, ST_Buffer(geom, 50) AS sommet
       FROM sn200_toponymes
       WHERE objectval LIKE ('%Gipfel')
    ) A
    JOIN vec200_cantons C ON ST_Intersects(A.sommet, C.geom)
    GROUP BY kantonsnr;

Pour ajouter le nombre de sommets à la table des cantons, nous modifions dans un premier temps la table des cantons pour y ajouter un champ pour le nombre de sommets:

    ALTER TABLE vec200_cantons ADD COLUMN nsommets INTEGER;

Puis dans un deuxième temps il est possible de mettre à jour cette table en utilisant la requête du haut comme sous-requête:

    UPDATE vec200_cantons
    SET nsommets = S.n
    FROM (
        SELECT kantonsnr, COUNT(*) AS n FROM
        (  SELECT gid, ST_Buffer(geom, 50) AS sommet
           FROM sn200_toponymes
           WHERE objectval LIKE ('%Gipfel')
        ) A
        JOIN vec200_cantons C ON ST_Intersects(A.sommet, C.geom)
        GROUP BY kantonsnr
    ) S
    WHERE vec200_cantons.kantonsnr = S.kantonsnr;

Nous pouvons désormais interroger la table des cantons pour connaître le résultat:

    SELECT * FROM vec200_cantons ORDER BY nsommets DESC;

Les numéros des cantons correspondent au numéros officiels donnés par l'OFS. On peut constater que les Grisons ont le plus de sommets devant le Valais et loin devant Berne (quelle surprise!). On peut aussi voir que 6 cantons n'ont pas de sommets sur la carte 1:200'000, dont Genève, Schaffhouse, Bâle et Argovie (encore une surprise!). On peut aussi calculer à partir de là que 55% des sommets suisses sont localisés dans les Grisons (30%) et le Valais (25%)...



## 12. Sauvegarde, import et export de bases de données

L'export et import de bases de données est faite en utilisant des fichiers SQL.

L'export de la base de données peut être faite depuis le Terminal avec:

    pg_dump --file=nom_du_fichier.sql nom_base_de_donnees


L'import de la base de données se fait simplement en utilisant l'utilitaire `psql`:

    psql nom_base_de_donnees < nom_du_fichier.sql

Le logiciel **pgAdmin III** propose également une option de sauvegarde d'une base de données ou d'une table (choisir le format *simple* / *plain*). L'import de la base de données est équivalent à l'exécution d'une série de requêtes SQL.